<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_04/TS%20-%20W4%20-%2012%20-%20Atividade_Avaliativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Usando dados do mundo real

Bem-vindo! Até agora, você trabalhou exclusivamente com dados sintéticos nas atividades avaliativas.

Desta vez, você usará o conjunto de dados [Daily Minimum Temperatures in Melbourne](https://github.com/jbrownlee/Datasets/blob/master/daily-min-temperatures.csv), que contém dados das temperaturas mínimas diárias registradas em Melbourne de 1981 a 1990.

Além de usar as camadas do Tensorflow para processar dados de sequência, como as camadas recorrentes ou LSTMs, você também usará as camadas convolucionais para melhorar o desempenho do modelo.

Vamos começar!

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/cert_prof_time_series/class_04/data.zip
!unzip -n -q data.zip

In [1]:
import csv
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from dataclasses import dataclass

Comece examinando a estrutura do arquivo csv que contém os dados:

In [ ]:
TEMPERATURES_CSV = './data/daily-min-temperatures.csv'

with open(TEMPERATURES_CSV, 'r') as csvfile:
    print(f"O cabeçalho tem a seguinte aparência:\n\n{csvfile.readline()}")    
    print(f"O primeiro ponto de dados tem a seguinte aparências:\n\n{csvfile.readline()}")
    print(f"O segundo ponto de dados tem a seguinte aparência:\n\n{csvfile.readline()}")

Como você pode ver, cada ponto de dados é composto pela data e pela temperatura mínima registrada para essa data.


No primeiro exercício, você codificará uma função para ler os dados do csv, mas, por enquanto, execute a próxima célula para carregar uma função auxiliar para plotar a série temporal.

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Tempo")
    plt.ylabel("Valor")
    plt.grid(True)

## Analisando os dados brutos
Agora você precisa ler os dados do arquivo csv. Para isso, complete a função `parse_data_from_file`.

Alguns aspectos a serem observados:

- Você deve omitir a primeira linha, pois o arquivo contém cabeçalhos.
- Não há necessidade de salvar os pontos de dados como matrizes numpy; listas regulares são suficientes.
- Para ler a partir de arquivos csv, use `csv.reader` passando os argumentos apropriados.
- O `csv.reader` retorna um iterável que retorna cada linha em cada iteração. Portanto, a temperatura pode ser acessada por meio de row[1] e a data pode ser descartada.
- A lista `times` deve conter cada intervalo de tempo (começando em zero), que é apenas uma sequência de números ordenados com o mesmo comprimento da lista `temperatures`.
- Os valores de `temperaturas` devem ser do tipo `float`. Você pode usar a função `float` integrada do Python para garantir isso.

In [ ]:
def parse_data_from_file(filename):
    
    times = []
    temperatures = []

    with open(filename) as csvfile:
        
        ### INICIE SEU CÓDGO AQUI
        
        reader = csv.reader(None, delimiter=None)
        
        ### TERMINE SEU CÓDGO AQUI
            
    return times, temperatures


A próxima célula usará sua função para calcular `times` e `temperatures` e os salvará como matrizes numpy dentro da classe de dados `G`. Essa célula também plotará a série temporal:

In [ ]:
# Teste sua função e salve todas as variáveis "globais" na classe G (G significa global)
@dataclass
class G:
    TEMPERATURES_CSV = './data/daily-min-temperatures.csv'
    times, temperatures = parse_data_from_file(TEMPERATURES_CSV)
    TIME = np.array(times)
    SERIES = np.array(temperatures)
    SPLIT_TIME = 2500
    WINDOW_SIZE = 64
    BATCH_SIZE = 32
    SHUFFLE_BUFFER_SIZE = 1000


plt.figure(figsize=(10, 6))
plot_series(G.TIME, G.SERIES)
plt.show()

**Saída Esperada:**
<div>
<img src="images/temp-series.png" width="500"/>
</div>

## Processamento dos dados

Como você já codificou as funções `train_val_split` e `windowed_dataset` durante as tarefas da semana passada, desta vez elas são fornecidas para você:

In [ ]:
def train_val_split(time, series, time_step=G.SPLIT_TIME):

    time_train = time[:time_step]
    series_train = series[:time_step]
    time_valid = time[time_step:]
    series_valid = series[time_step:]

    return time_train, series_train, time_valid, series_valid


# Dividir o conjunto de dados
time_train, series_train, time_valid, series_valid = train_val_split(G.TIME, G.SERIES)

In [ ]:
def windowed_dataset(series, window_size=G.WINDOW_SIZE, batch_size=G.BATCH_SIZE, shuffle_buffer=G.SHUFFLE_BUFFER_SIZE):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    ds = ds.batch(batch_size).prefetch(1)
    return ds


# Aplicar a transformação ao conjunto de treinamento
train_set = windowed_dataset(series_train, window_size=G.WINDOW_SIZE, batch_size=G.BATCH_SIZE, shuffle_buffer=G.SHUFFLE_BUFFER_SIZE)

## Definição da arquitetura do modelo

Agora que você tem uma função que processará os dados antes que eles sejam inseridos na rede neural para treinamento, é hora de definir a arquitetura da camada. Assim como na tarefa da semana passada, você fará a definição e a compilação da camada em duas etapas separadas. Comece completando a função `create_uncompiled_model` abaixo.

Isso é feito para que você possa reutilizar as camadas do seu modelo para o ajuste da taxa de aprendizado e o treinamento real.

Dica:

- As camadas `Lambda` não são necessárias.
- Use uma combinação de camadas `Conv1D` e `LSTM` seguidas de camadas `Dense`

In [ ]:
def create_uncompiled_model():

    ### INICIE SEU CÓDGO AQUI
    
    model = tf.keras.models.Sequential([
        
    ]) 
    
    ### TERMINE SEU CÓDGO AQUI

    return model

In [ ]:
# Teste seu modelo não compilado
uncompiled_model = create_uncompiled_model()

try:
    uncompiled_model.predict(train_set)
except:
    print("Sua arquitetura atual é incompatível com o conjunto de dados em janela, tente ajustá-la.")
else:
    print("Sua arquitetura atual é compatível com o conjunto de dados com janelas! :)")

## Ajuste da taxa de aprendizado

Como você viu na aula, é possível aproveitar os _callbacks_ do Tensorflow para variar dinamicamente a taxa de aprendizagem durante o treinamento. Isso pode ser útil para ter uma noção melhor de qual taxa de aprendizado se adapta melhor ao problema em questão.

**Observe que isso só altera a taxa de aprendizagem durante o processo de treinamento para lhe dar uma ideia de qual é uma taxa de aprendizagem razoável e não deve ser confundido com a seleção da melhor taxa de aprendizagem, o que é conhecido como otimização de hiperparâmetros.**

Para os otimizadores, você pode experimentar:

- tf.keras.optimizers.Adam
- tf.keras.optimizers.SGD com um momentum de 0,9

In [ ]:
def adjust_learning_rate(dataset):
    
    model = create_uncompiled_model()
    
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
    
    ### INICIE SEU CÓDIGO AQUI
    
    # Selecione seu otimizador
    optimizer = None

    # Compile o modelo passando a perda apropriadas
    model.compile(loss=None,
                  optimizer=optimizer, 
                  metrics=["mae"]) 
    
    ### TERMINE SEU CÓDIGO AQUI
    
    history = model.fit(dataset, epochs=100, callbacks=[lr_schedule])
    
    return history

In [ ]:
# Executar o treinamento com LR dinâmico
lr_history = adjust_learning_rate(train_set)

In [ ]:
plt.semilogx(lr_history.history["lr"], lr_history.history["loss"])
plt.axis([1e-4, 10, 0, 10])

## Compilação do modelo
Agora que você treinou o modelo variando a taxa de aprendizado, é hora de fazer o treinamento real que será usado para prever a série temporal. Para isso, complete a função `create_model` abaixo.

Observe que você está reutilizando a arquitetura definida anteriormente em `create_uncompiled_model`. Agora você só precisa compilar esse modelo usando a perda, o otimizador (e a taxa de aprendizado) apropriados.

Dicas:

- O treinamento deve ser muito rápido, portanto, se perceber que cada época está demorando mais do que alguns segundos, considere tentar uma arquitetura diferente.


- Se, após a primeira época, você obtiver uma saída como esta: loss: nan - mae: nan, é muito provável que sua rede esteja sofrendo com a explosão de gradientes. Esse é um problema comum se você usou o SGD como otimizador e definiu uma taxa de aprendizado muito alta. Se você encontrar esse problema, considere reduzir a taxa de aprendizado ou usar o Adam com a taxa de aprendizado padrão.

In [ ]:
def create_model():

    
    model = create_uncompiled_model()

    ### INICIE SEU CÓDIGO AQUI

    model.compile(loss=None,
                  optimizer=None,
                  metrics=["mae"])  
    

    ### FINALIZE SEU CÓDIGO AQUI

    return model

In [ ]:
# Salvar uma instância do modelo
model = create_model()

# Treine-o
history = model.fit(train_set, epochs=50)

## Avaliação da previsão

Agora é hora de avaliar o desempenho da previsão. Para isso, você pode usar a função `compute_metrics` que codificou em uma tarefa anterior:

In [ ]:
def compute_metrics(true_series, forecast):
    
    mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
    mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

    return mse, mae

Neste ponto, apenas o modelo que realizará a previsão está pronto, mas você ainda precisa calcular a previsão real.


## Previsões de modelo mais rápidas

Anteriormente, você viu uma abordagem mais rápida em comparação com o uso de um loop for para calcular as previsões para cada ponto da sequência. Lembre-se de que essa abordagem mais rápida usa lotes de dados. 

O código para implementar isso é fornecido no `model_forecast` abaixo. Observe que o código é muito semelhante ao da função `windowed_dataset` com a diferença de que:
- O conjunto de dados é janelado usando `window_size` em vez de `window_size + 1`
- Não deve ser usado shuffle
- Não há necessidade de dividir os dados em recursos e rótulos
- Um modelo é usado para prever lotes do conjunto de dados

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

Agora calcule a previsão real:

**Nota:** Não modifique a célula abaixo. 

O avaliador usa o mesmo fatiamento para obter a previsão, portanto, se você alterar a célula abaixo, poderá ter problemas ao enviar o modelo para avaliação.

In [ ]:
# Calcular a previsão para todas as séries
rnn_forecast = model_forecast(model, G.SERIES, G.WINDOW_SIZE).squeeze()

# Corte a previsão para obter apenas as previsões para o conjunto de validação
rnn_forecast = rnn_forecast[G.SPLIT_TIME - G.WINDOW_SIZE:-1]

# Plotar a previsão
plt.figure(figsize=(10, 6))
plot_series(time_valid, series_valid)
plot_series(time_valid, rnn_forecast)

In [ ]:
mse, mae = compute_metrics(series_valid, rnn_forecast)

print(f"mse: {mse:.2f}, mae: {mae:.2f} for forecast")

**Para ser aprovado nesta tarefa, sua previsão deve atingir um MSE de 6 ou menos e um MAE de 2 ou menos.**

- Se a sua previsão não atingir esse limite, tente treinar novamente o modelo com uma arquitetura diferente (será necessário executar novamente as funções `create_uncompiled_model` e `create_model`) ou ajustar os parâmetros do otimizador.


- Se a sua previsão atingiu esse limite, execute a célula a seguir para salvar o modelo no formato SavedModel, que será usado para avaliação e, depois disso, envie sua tarefa para avaliação.


- Esse ambiente inclui um diretório SavedModel fictício que contém um modelo fictício treinado para uma época. **Para substituir esse arquivo pelo seu modelo real, você precisa executar a próxima célula antes de enviá-lo para avaliação.**

In [3]:
# Salve seu modelo no formato SavedModel
model.save('saved_model/my_model')

# Comprimir o diretório usando tar
! tar -czvf saved_model.tar.gz saved_model/

**Parabéns por terminar a atividade avaliativa desse módulo!

Você implementou com sucesso uma rede neural capaz de prever séries temporais aproveitando uma combinação de camadas do Tensorflow, como Convolutional e LSTMs! Isso resultou em uma previsão que supera todas as que você fez anteriormente.